In [2]:
import os
import requests
import pandas as pd
import random
from typing import Dict, List
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [3]:
load_dotenv()
PG_DSN = (
    f"postgresql://{os.getenv('PG_USER')}:"
    f"{os.getenv('PG_PASS')}@"
    f"{os.getenv('PG_HOST')}:"
    f"{os.getenv('PG_PORT')}/"
    f"{os.getenv('PG_DB')}"
)
OPENROUTER_KEY = os.getenv('OPENROUTER_KEY')
engine = create_engine(PG_DSN)

In [ ]:
from utils import fetch_random_news

class GPTSynthesizer:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.url = "https://openrouter.ai/api/v1/chat/completions"

    def _gpt_query(self, system_prompt: str, prompt: str) -> str:
        messages: List[Dict[str, str]] = []
        if system_prompt:
            messages.append({
                "role": "system",
                "content": "Всегда отвечай на русском языке. " + system_prompt
            })
        messages.append({"role": "user", "content": prompt})

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

        payload = {
            "model": "qwen/qwen-2.5-72b-instruct",
            "messages": messages,
            "temperature": 0,
            "max_tokens": 1024,
            "seed": 42,
        }

        response = requests.post(self.url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

    def generate_for_article(self, title: str, anons: str, body: str) -> Dict[str, str]:
        """
        Для переданной статьи (title, anons, body) формирует 3 поисковых запроса
        с разными уровнями сложности: лёгкий, средний, сложный.
        Возвращает словарь {"easy": ..., "medium": ..., "hard": ...}.
        """
        # Собираем контекст: комбинируем title + anons (если есть) + первые 200 символов body
        snippet = body[:200].replace("\n", " ")
        full_context = f"Заголовок: {title}\nАнонс: {anons or ''}\nТело (фрагмент): {snippet}..."
        system_prompt = (
            "Ты генерируешь три поисковых запроса разного уровня сложности, "
            "чтобы найти именно эту новость на новостном портале. Важно, чтобы запрос был именно к этой новости"
            "Первый запрос — максимально буквальный, в точности по ключевым словам. "
            "Второй — чуть более обобщённый (с синонимами). "
            "Третий — максимально абстрактный, где используются контекстные формулировки и косвенные признаки, возможны реалестичные опечатки, чтобы тестировать поисковую систему по новостям"
        )
        prompt = (
            f"{full_context}\n\n"
            "Сгенерируй три отдельных строки, помеченные как:\n"
            "1) Лёгкий запрос: <здесь>\n"
            "2) Средний запрос: <здесь>\n"
            "3) Сложный запрос: <здесь>"
        )
        response = self._gpt_query(system_prompt, prompt)
        lines = [line.strip() for line in response.split("\n") if line.strip()]
        result = {"easy": "", "medium": "", "hard": ""}
        for line in lines:
            if line.startswith("1") or line.lower().startswith("1)"):
                result["easy"] = line.split(":", 1)[1].strip()
            elif line.startswith("2") or line.lower().startswith("2)"):
                result["medium"] = line.split(":", 1)[1].strip()
            elif line.startswith("3") or line.lower().startswith("3)"):
                result["hard"] = line.split(":", 1)[1].strip()
        return result

In [ ]:
df_sample = fetch_random_news(limit=100, sample_pct=1.0)
gpt = GPTSynthesizer(api_key=OPENROUTER_KEY)


In [12]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from tqdm.autonotebook import tqdm


async def _process_row(executor, row):
    art_id = row["id"]
    title, anons, body = row["title"], row["anons"], row["body"]

    gens = await asyncio.get_running_loop().run_in_executor(
        executor, lambda: gpt.generate_for_article(title, anons, body)
    )
    return art_id, gens

async def generate_synthetic(df):
    synthetic = []
    with ThreadPoolExecutor() as executor:
        tasks = [
            asyncio.create_task(_process_row(executor, row))
            for _, row in df.iterrows()
        ]
        for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            art_id, gens = await coro
            synthetic.append((art_id, "easy",   gens["easy"]))
            synthetic.append((art_id, "medium", gens["medium"]))
            synthetic.append((art_id, "hard",   gens["hard"]))
    return synthetic


synthetic_queries = await generate_synthetic(df_sample)

100%|██████████| 100/100 [01:01<00:00,  1.63it/s]


In [13]:
df_synth = pd.DataFrame(synthetic_queries, columns=["article_id", "difficulty", "query"])
queries_gt: Dict[str, List[str]] = {}
for art_id, difficulty, q in tqdm(synthetic_queries):
    queries_gt[q] = [art_id]

100%|██████████| 300/300 [00:00<00:00, 1459734.57it/s]


In [14]:
df_synth.to_csv("synthetic_queries.csv", index=False)